In [1]:
# Data Manipulation and Analysis
import numpy as np 
import pandas as pd 

# Geographic Data Handling
import geopandas as gpd 
from geopandas.tools import sjoin
from vt2geojson.tools import vt_bytes_to_geojson  
from shapely.geometry import Point, box
from scipy.spatial import cKDTree 
import osmnx as ox  
import mercantile  

# File and Directory Operations
import os 

# Image Processing and Analysis
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation  
from scipy.signal import find_peaks 
import torch  
from PIL import Image  
import requests  

# Libraries for Concurrency and File Manipulation
from concurrent.futures import ThreadPoolExecutor, as_completed 
import multiprocessing as mp 

# Time and Date Handling
from time import time
from datetime import timedelta

# Progress Tracking
from tqdm import tqdm  

In [2]:
# Test data
test_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/"
test_results_path = "C:/Users/ygrin/Documents/Studie - MSc ADS/Utrecht University/Block 4 - Thesis/TestData/Results/"
single_point = gpd.read_file(test_path + "Test_single_home_location.gpkg")
multi_point = gpd.read_file(test_path + "Test_multiple_home_locations.gpkg")
polygon = gpd.read_file(test_path + "test_polygon.gpkg")

# USER NETWORK
network = gpd.read_file(test_path + "test_network_shp.shp").to_crs('EPSG:27700')

# GVI functions

Original code from Ilse A. Vázquez Sánchez: https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/StreetView-NatureVisibility

In [3]:
def get_road_network_with_points(poi_polygon, epsg):
    # Get the road network within the bounding box
    G = ox.graph_from_polygon(poi_polygon, network_type='drive', simplify=True)

    #Project the graph from latitude-longitude coordinates to a local projection (in meters)
    G_proj = ox.project_graph(G, to_crs=f"EPSG:{epsg}")

    # Convert the projected graph to a GeoDataFrame
    edges = ox.graph_to_gdfs(G_proj, nodes=False, edges=True)

    return edges


# Get a list of points over the road map with a N distance between them
def select_points_on_road_network(network_edges, distance=50):
    # Initialize a list to store the points
    points = []

    # Loop through each road in the road network graph
    for road in network_edges.geometry:
        # Calculate the total length of the road
        road_length = road.length

        # Start at the beginning of the road
        current_position = 0

        # Loop through the road, adding points every 50 meters
        while current_position < road_length:
            # Get the point on the road at the current position
            current_point = road.interpolate(current_position)

            # Add the curent point to the list of points
            points.append(current_point)

            # Increment the position by the desired distance
            current_position += distance
    
    # Convert the list of points to a GeoDataFrame
    gdf_points = gpd.GeoDataFrame(geometry=points, crs=network_edges.crs)

    return gdf_points


def select_points_within_buffers(poi, road_points):
    points_within_buffers = sjoin(road_points, poi.set_geometry('buffer'), how='inner', predicate='within')

    # Get the unique points that fall within any buffer
    unique_points = points_within_buffers['geometry_left'].unique()

    # Create a new GeoDataFrame with the points that fall within any buffer
    return gpd.GeoDataFrame(geometry=[Point(p.x, p.y) for p in unique_points], crs=poi.crs)


def get_features_for_tile(tile, access_token):
    tile_url = f"https://tiles.mapillary.com/maps/vtp/mly1_public/2/{tile.z}/{tile.x}/{tile.y}?access_token={access_token}"
    response = requests.get(tile_url)
    result = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z, layer="image")
    return [tile, result]


def get_features_on_points(buffer_points, access_token, zoom=14):
    # Transform the coordinate reference system to EPSG 4326
    buffer_points_wgs = buffer_points.copy(deep=True).to_crs(epsg=4326)

    # Add a new column to gdf_points that contains the tile coordinates for each point
    buffer_points['tile'] = [mercantile.tile(x, y, zoom) for x, y in zip(buffer_points_wgs.geometry.x, buffer_points_wgs.geometry.y)]

    # Group the points by their corresponding tiles
    groups = buffer_points.groupby('tile')

    # Download the tiles and extract the features for each group
    features = []
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []

        for tile, _ in groups:
            futures.append(executor.submit(get_features_for_tile, tile, access_token))
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading tiles"):
            result = future.result()
            features.append(result)

    pd_features = pd.DataFrame(features, columns=["tile", "features"])

    # Compute distances between each feature and all the points in gdf_points
    feature_points = pd.DataFrame(
        [(Point(f["geometry"]["coordinates"]), f) for row in pd_features["features"] for f in row["features"]],
        columns=["geometry", "feature"]
    )
    feature_tree = cKDTree(feature_points["geometry"].apply(lambda p: [p.x, p.y]).tolist())
    _, indices = feature_tree.query(buffer_points_wgs["geometry"].apply(lambda p: [p.x, p.y]).tolist())

    # Select the closest feature for each point
    buffer_points["feature"] = feature_points.loc[indices, "feature"].tolist()

    # Convert results to geodataframe
    buffer_points['tile'] = buffer_points['tile'].astype(str)
    
    return buffer_points


def get_models():
    processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")
    model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")
    return processor, model


def run_length_encoding(in_array):
    image_array = np.asarray(in_array)
    length = len(image_array)
    if length == 0: 
        return (None, None, None)
    else:
        pairwise_unequal = image_array[1:] != image_array[:-1]
        change_points = np.append(np.where(pairwise_unequal), length - 1)   # must include last element posi
        run_lengths = np.diff(np.append(-1, change_points))       # run lengths
        return(run_lengths, image_array[change_points])


def get_road_pixels_per_column(prediction):
    road_pixels = prediction == 0.0 # The label for the roads is 0
    road_pixels_per_col = np.zeros(road_pixels.shape[1])
    
    for i in range(road_pixels.shape[1]):
        run_lengths, values = run_length_encoding(road_pixels[:,i])
        road_pixels_per_col[i] = run_lengths[values.nonzero()].max(initial=0)
    return road_pixels_per_col


def get_road_centres(prediction, distance=2000, prominence=100):
    road_pixels_per_col = get_road_pixels_per_column(prediction)
    peaks, _ = find_peaks(road_pixels_per_col, distance=distance, prominence=prominence)
    
    return peaks


def find_road_centre(segmentation):
	distance = int(2000 * segmentation.shape[1] // 5760)
	prominence = int(100 * segmentation.shape[0] // 2880)
	
	centres = get_road_centres(segmentation, distance=distance, prominence=prominence)
	
	return centres


def crop_panoramic_images(original_width, image, segmentation, road_centre):
    width, height = image.size

    # Find duplicated centres
    duplicated_centres = [centre - original_width for centre in road_centre if centre >= original_width]
            
    # Drop the duplicated centres
    road_centre = [centre for centre in road_centre if centre not in duplicated_centres]

    # Calculate dimensions and offsets
    w4 = int(width / 4) # 
    h4 = int(height / 4)
    hFor43 = int(w4 * 3 / 4)
    w98 = width + (w4 / 2)
    xrapneeded = int(width * 7 / 8)

    images = []
    pickles = []
    # Crop the panoramic image
    for centre in road_centre:
        # Wrapped all the way around
        if centre >= w98:
            xlo = int(centre - w4/2)
            cropped_image = image.crop((xlo, h4,  xlo+w4, h4 + hFor43))
            cropped_segmentation = segmentation[h4:h4+hFor43, xlo:xlo+w4]
        
        # Image requires assembly of two sides
        elif centre > xrapneeded:
            xlo = int(centre - (w4/2)) # horizontal_offset
            w4_p1 = width - xlo
            w4_p2 = w4 - w4_p1
            cropped_image_1 = image.crop((xlo, h4, xlo + w4_p1, h4 + hFor43))
            cropped_image_2 = image.crop((0, h4, w4_p2, h4 + hFor43))

            cropped_image = Image.new(image.mode, (w4, hFor43))
            cropped_image.paste(cropped_image_1, (0, 0))
            cropped_image.paste(cropped_image_2, (w4_p1, 0))

            cropped_segmentation_1 = segmentation[h4:h4+hFor43, xlo:xlo+w4_p1]
            cropped_segmentation_2 = segmentation[h4:h4+hFor43, 0:w4_p2]
            cropped_segmentation = torch.cat((cropped_segmentation_1, cropped_segmentation_2), dim=1)
        
        # Must paste together the two sides of the image
        elif centre < (w4 / 2):
            w4_p1 = int((w4 / 2) - centre)
            xhi = width - w4_p1
            w4_p2 = w4 - w4_p1

            cropped_image_1 = image.crop((xhi, h4, xhi + w4_p1, h4 + hFor43))
            cropped_image_2 = image.crop((0, h4, w4_p2, h4 + hFor43))

            cropped_image = Image.new(image.mode, (w4, hFor43))
            cropped_image.paste(cropped_image_1, (0, 0))
            cropped_image.paste(cropped_image_2, (w4_p1, 0))

            cropped_segmentation_1 = segmentation[h4:h4+hFor43, xhi:xhi+w4_p1]
            cropped_segmentation_2 = segmentation[h4:h4+hFor43, 0:w4_p2]
            cropped_segmentation = torch.cat((cropped_segmentation_1, cropped_segmentation_2), dim=1)
            
        # Straightforward crop
        else:
            xlo = int(centre - w4/2)
            cropped_image = image.crop((xlo, h4, xlo + w4, h4 + hFor43))
            cropped_segmentation = segmentation[h4:h4+hFor43, xlo:xlo+w4]
        
        images.append(cropped_image)
        pickles.append(cropped_segmentation)

    return images, pickles


def segment_images(image, processor, model):
    inputs = processor(images=image, return_tensors="pt")
    
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # You can pass them to processor for postprocessing
    segmentation = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]

    return segmentation


def get_GVI(segmentations):
    green_percentage = 0
    for segment in segmentations:
        total_pixels = segment.numel()
        vegetation_pixels = (segment == 8).sum().item()
        green_percentage += vegetation_pixels / total_pixels
    
    return green_percentage / len(segmentations)


def process_images(image_url, is_panoramic, processor, model):
    try:
        image = Image.open(requests.get(image_url, stream=True).raw)

        if is_panoramic:
            # Get the size of the image
            width, height = image.size

            # Crop the bottom 20% of the image to cut the band on the bottom of the panoramic image
            bottom_crop = int(height * 0.2)
            image = image.crop((0, 0, width, height - bottom_crop))

        # Image segmentation
        segmentation = segment_images(image, processor, model)

        if is_panoramic:
            # Create a widened panorama by wrapping the first 25% of the image onto the right edge
            width, height = image.size
            w4 = int(0.25 * width)

            segmentation_25 = segmentation[:, :w4]
            # Concatenate the tensors along the first dimension (rows)
            segmentation_road = torch.cat((segmentation, segmentation_25), dim=1)
        else:
            segmentation_road = segmentation
        
        # Find roads to determine if the image is suitable for the analysis or not AND crop the panoramic images
        road_centre = find_road_centre(segmentation_road)

        if len(road_centre) > 0:
            if is_panoramic:
                images, pickles = crop_panoramic_images(width, image, segmentation_road, road_centre)
            else:
                images = [image]
                pickles = [segmentation]
        
            # Now we can get the Green View Index
            GVI = get_GVI(pickles)
            return [GVI, is_panoramic, False, False]
        else:
            # There are not road centres, so the image is unusable
            return [None, None, True, False]
    except:
        return [None, None, True, True]


def download_image(geometry, image_metadata, access_token, processor, model):
    header = {'Authorization': 'OAuth {}'.format(access_token)}

    image_id = image_metadata["properties"]["id"]
    is_panoramic = image_metadata["properties"]["is_pano"]
    
    url = 'https://graph.mapillary.com/{}?fields=thumb_original_url'.format(image_id)
    response = requests.get(url, headers=header)
    data = response.json()
    image_url = data["thumb_original_url"]

    result = process_images(image_url, is_panoramic, processor, model)
    result.insert(0, geometry)

    return result


def process_data(gdf, access_token, epsg, max_workers=1):
    processor, model = get_models()

    gdf_wgs = gdf.copy(deep=True).to_crs("EPSG:4326")

    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, row in gdf_wgs.iterrows():
            feature = row["feature"]
            geometry = row["geometry"]
            futures.append(executor.submit(download_image, geometry, feature, access_token, processor, model))
    
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Downloading images"):
            image_result = future.result()
            results.append(image_result)
    
    images_results_gdf = gpd.GeoDataFrame(results, columns=["geometry", "GVI", "is_panoramic", "missing", "error"], crs="EPSG:4326").to_crs(f"EPSG:{epsg}")
    return images_results_gdf
    
'''
def process_data(index, data_part, processor, model, access_token, max_workers):
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, row in data_part.iterrows():
            feature = row["feature"]
            geometry = row["geometry"]
            futures.append(executor.submit(download_image, geometry, feature, access_token, processor, model))
    
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Downloading images (Process {index})"):
            image_result = future.result()
            results.append(image_result)
        return results

def process_data(gdf, access_token, epsg, max_workers):
    gdf_wgs = gdf.copy(deep=True).to_crs("EPSG:4326")
    processor, model = get_models()
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, row in gdf_wgs.iterrows():
            feature = row["feature"]
            geometry = row["geometry"]
            futures.append(executor.submit(download_image, geometry, feature, access_token, processor, model))
    
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Downloading images"):
            image_result = future.result()
            results.append(image_result)

    images_results_gdf = gpd.GeoDataFrame(results, columns=["geometry", "GVI", "is_panoramic", "missing", "error"], crs="EPSG:4326").to_crs(f"EPSG:{epsg}")
    return images_results_gdf

def download_images_for_points(gdf, access_token, epsg, max_workers=1):
    processor, model = get_models()
    
    gdf_wgs = gdf.copy(deep=True).to_crs("EPSG:4326")

    images_results = []

    # Split the dataset into parts
    num_processes = mp.cpu_count()-1 # Get the number of CPU cores
    data_parts = np.array_split(gdf_wgs, num_processes) # Split the dataset
    
    with mp.get_context("spawn").Pool(processes=num_processes) as pool:
        # Apply the function to each part of the dataset using multiprocessing
        results = pool.starmap(process_data, [(index, data_part, processor, model, access_token, max_workers) for index, data_part in enumerate(data_parts)])

        # Combine the results from all parts
        images_results = [result for part_result in results for result in part_result]

        # Close the pool to release resources
        pool.close()
        pool.join()

    images_results_gdf = gpd.GeoDataFrame(images_results, columns=["geometry", "GVI", "is_panoramic", "missing", "error"], crs="EPSG:4326").to_crs(f"EPSG:{epsg}")
    return images_results_gdf
'''


def get_gvi_per_buffer(buffered_points, gvi_per_point):
    joined = gpd.sjoin(gvi_per_point, buffered_points.set_geometry('buffer'), how='inner', predicate='within').drop('index_right', axis=1)
    
    # Group the points by buffer
    grouped = joined.groupby('id', group_keys=True)
    # Convert 'grouped' to a DataFrame
    grouped_df = grouped.apply(lambda x: x.reset_index(drop=True))
    grouped_df = grouped_df[["geometry_left", "GVI", "is_panoramic", "missing"]].reset_index()
    # Convert grouped_df to a GeoDataFrame
    grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry_left').rename(columns={'geometry_left':'geometry'}).drop('level_1', axis=1)
    grouped_gdf = grouped_gdf.set_geometry('geometry')
    # Calculate the average 'gvi' for each group
    avg_gvi = np.round(grouped['GVI'].mean(),3).reset_index()
    nr_of_points = grouped['GVI'].count().reset_index(name='nr_of_points')
    # Merge with the buffered_points dataframe to get the buffer geometries
    result = avg_gvi.merge(buffered_points, left_on='id', right_on='id')
    result = result.merge(nr_of_points, on='id')
    # Convert the result to a GeoDataFrame
    result = gpd.GeoDataFrame(result[['id', 'geometry', 'GVI', 'nr_of_points']])

    return result, grouped_gdf

# Top level function

In [4]:
def get_streetview_GVI(point_of_interest_file, access_token=None, crs_epsg=None, polygon_type="neighbourhood", buffer_dist=None,
                       network_file=None, write_to_file=True, output_dir=os.getcwd()):
    
    ### Step 1: Read and process user inputs, check conditions
    poi = gpd.read_file(point_of_interest_file)
    # Make sure geometries of poi file are either all provided using point geometries or all using polygon geometries
    if all(poi['geometry'].geom_type == 'Point') or all(poi['geometry'].geom_type == 'Polygon'):
        geom_type = poi.iloc[0]['geometry'].geom_type
    else:
        raise ValueError("Please make sure all geometries are of 'Point' type or all geometries are of 'Polygon' type and re-run the function")

    # Make sure CRS of poi file is projected rather than geographic
    if not poi.crs.is_projected:
        if crs_epsg is None:
            print("Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to CRS with EPSG:3395")
            epsg = 3395
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
        else:
            print(f"Warning: The CRS of the PoI dataset is currently geographic, therefore it will now be projected to EPSG:{crs_epsg} as specified")
            epsg = crs_epsg
            poi.to_crs(f"EPSG:{epsg}", inplace=True)
    else:
        epsg = poi.crs.to_epsg()

    # In case of house polygons, transform to centroids
    if geom_type == "Polygon":
        if polygon_type not in ["neighbourhood", "house"]:
            raise TypeError("Please make sure that the polygon_type argument is set to either 'neighbourhood' or 'house'")
        if polygon_type == "house":
            print("Changing geometry type to Point by computing polygon centroids...")
            poi['geometry'] = poi['geometry'].centroid
            geom_type = poi.iloc[0]['geometry'].geom_type
            print("Done \n")

    # Make sure poi dataframe has id column
    if "id" in poi.columns:
        if poi['id'].isnull().values.any():
            poi['id'] = poi['id'].fillna(pd.Series(range(1, len(poi) + 1))).astype(int)
    else:
        poi['id'] = pd.Series(range(1, len(poi) + 1)).astype(int)

    # Make sure buffer distance is set in case geometries of poi file are of point type
    if geom_type == "Point":
        if not isinstance(buffer_dist, int) or (not buffer_dist > 0):
            raise TypeError("Please make sure that the buffer_dist argument is set to a positive integer")

    # Make sure Mapillary API token is provided
    if access_token is None:
        raise TypeError("Please make sure that an access token for Mapillary is provided")
    
    # Determine area of interest by taking bounding box of poi file, incl. buffer if specified
    if buffer_dist is None:
        poi_polygon = box(*poi.total_bounds)
        poi['buffer'] = poi['geometry']
    else:
        poi_polygon = box(*poi.total_bounds).buffer(buffer_dist)
        poi['buffer'] = poi['geometry'].buffer(buffer_dist)
    # Transform to 4326 for OSM
    polygon_gdf_wgs = gpd.GeoDataFrame(geometry=[poi_polygon], crs=f"EPSG:{epsg}").to_crs("EPSG:4326") 
    # Extract polygon in EPSG 4326
    wgs_polygon = polygon_gdf_wgs['geometry'].values[0] 

    if network_file is not None:
        # Make sure network file is either provided as geopackage or shapefile
        if os.path.splitext(network_file)[1] not in [".gpkg", ".shp"]:
            raise ValueError("Please provide the network file in '.gpkg' or '.shp' format")
        elif network_file is not None and (os.path.splitext(network_file)[1] == ".gpkg"):
            network_edges = gpd.read_file(network_file, layer='edges')
        else: 
            network_edges = gpd.read_file(network_file)

        # Make sure network file CRS is same as CRS of poi file
        if not network_edges.crs.to_epsg() == epsg:
            print("Adjusting CRS of Network file to match with Point of Interest CRS...")
            network_edges.to_crs(f'EPSG:{epsg}', inplace=True)
            print("Done \n")

        # Check if house locations are within network file provided
        bbox_network = network_edges.unary_union.envelope
        if not all(geom.within(bbox_network) for geom in poi['geometry']):
            raise ValueError("Not all points of interest are within the network file provided, please make sure they are and re-run the function")
    else:
        print(f"Retrieving network within total bounds of {geom_type}(s) of interest, extended by the buffer_dist in case provided...")
        start_network_retrieval = time()
        # Extract network from OpenStreetMap
        network_edges = get_road_network_with_points(wgs_polygon, epsg=epsg)
        end_network_retrieval = time()
        elapsed_network_retrieval = end_network_retrieval - start_network_retrieval
        print(f"Done, running time: {str(timedelta(seconds=elapsed_network_retrieval))} \n")

    print("Computing sample points for roads within area of interest's network...")
    start_sample_points = time()
    # Get sample points on network roads
    road_points = select_points_on_road_network(network_edges)
    # Filter points to maintain the ones that are within the buffers of the poi geometries
    buffer_points = select_points_within_buffers(poi, road_points)
    end_sample_points = time()
    elapsed_sample_points = end_sample_points - start_sample_points
    print(f"Done, running time: {str(timedelta(seconds=elapsed_sample_points))} \n")
    
    print("Downloading StreetView images for road sample points...")
    start_images = time()
    # Retrieve features, images, from Mapillary for the road locations
    features = get_features_on_points(buffer_points, access_token)
    #gvi_per_point = download_images_for_points(features, access_token, epsg)
    # Process the features and calculate GVI score for road locations
    gvi_per_point = process_data(features, access_token, epsg)
    end_images = time()
    elapsed_images = end_images - start_images
    print(f"Done, running time: {str(timedelta(seconds=elapsed_images))} \n")
    
    print("Calculating StreetView GVI score...")
    start_calc = time()
    # Calculate average GVI for each geometry in poi dataframe
    poi, sampled_points_gdf = get_gvi_per_buffer(poi, gvi_per_point)
    end_calc = time()
    elapsed_calc = end_calc - start_calc
    print(f"Done, running time: {str(timedelta(seconds=elapsed_calc))} \n")

    print("Note: workflow for calculating Streetview GVI based on code by Ilse A. Vázquez Sánchez \nsource: https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/StreetView-NatureVisibility \n")
    
    if write_to_file:
        print("Writing results to new geopackage file in specified directory...")
        # Create output directory if the one specified by user does not yet exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        # Extract filename of poi file to add information when writing to file
        input_filename, _ = os.path.splitext(os.path.basename(point_of_interest_file))
        poi.to_file(os.path.join(output_dir, f"{input_filename}_StreetviewGVI_added.gpkg"), driver="GPKG")
        sampled_points_gdf.to_file(os.path.join(output_dir, f"{input_filename}_StreetviewGVI_sampled_points.gpkg"), driver="GPKG")
        print("Done")

    return poi, sampled_points_gdf

In [5]:
poi, sampled_points_gdf = get_streetview_GVI(point_of_interest_file=test_path+"Test_multiple_home_locations.gpkg",
                                             access_token="MAPILLARY_API_TOKEN", 
                                             crs_epsg=None, 
                                             polygon_type="neighbourhood", 
                                             buffer_dist=65, 
                                             write_to_file=True, 
                                             output_dir=test_results_path)

Retrieving network within total bounds of Point(s) of interest, extended by the buffer_dist in case provided...
Done, running time: 0:00:04.616102 

Computing sample points for roads within area of interest's network...
Done, running time: 0:00:00.522553 



Done, running time: 0:06:40.280089 

Calculating StreetView GVI score...
Done, running time: 0:00:00.151857 

Note: workflow for calculating Streetview GVI based on code by Ilse A. Vázquez Sánchez 
source: https://github.com/Spatial-Data-Science-and-GEO-AI-Lab/StreetView-NatureVisibility 

Writing results to new geopackage file in specified directory...
Done


In [6]:
poi

,id,geometry,GVI,nr_of_points
0,2,POINT (385981.911 393805.494),0.324,3
1,3,POINT (388631.230 395322.181),0.013,8


In [7]:
sampled_points_gdf

,id,geometry,GVI,is_panoramic,missing
0,2,POINT (385990.438 393837.957),0.323574,False,False
1,2,POINT (385988.779 393855.628),0.323574,False,False
2,2,POINT (385988.752 393837.984),0.323574,False,False
3,3,POINT (388654.098 395336.215),0.013628,False,False
4,3,POINT (388653.274 395286.222),0.014712,False,False
5,3,POINT (388645.478 395379.148),0.037419,False,False
6,3,POINT (388694.240 395336.063),NaN,None,True
7,3,POINT (388612.771 395334.892),0.010661,False,False
8,3,POINT (388590.281 395297.803),0.001739,False,False
9,3,POINT (388653.594 395305.607),0.014712,False,False
